# Análisis de campañas de alivio financiero y propuesta para recuperación de cartera vencida en una entidad financiera

## Breve descripción del proyecto

En el presente proyecto, se realiza el análisis de los datos correspondientes a una campaña que realiza la entidad financiera, a la cual se la referirá mediante el nombre ficticio ***Mi Banco Feliz***, cuyo objetivo es lanzar 2 campañas en pro de sus clientes en estado de mora a fin de recuperar más de un 60% de la cartera vencida que se mantiene con corte a la fecha de este proyecto, es decir, Junio de 2025.

Se realiza este análisis debido a que las campañas de alivio financiero, llamadas diferimiento y normalización, han tenido baja efectividad en cuanto a recuperación monetaria frente a las metas previamente establecidas por el departamento de cobranzas de la Entidad.

Se van a responder a las siguientes preguntas para poder entender de mejor manera cuál es el problema actual y de que manera se podría brindar una solución que satizfaga tanto las necesidades del Banco sin descuidar el bienestar del cliente:

1. ¿Qué perfiles de clientes están siendo atendidos por cada campaña y canal?
2. ¿Cómo podría optimizarse el enfoque actual para mejorar la efectividad de las campañas?

**Nota:** *A lo largo del presente reporte se analizan datos reales, mismos que han sido debidamente ofuscados y/o omitidos intencionalmente con el fin de proteger posible información sensible, así como por mantener respeto y apego hacia las leyes de protección de datos tanto locales como internacionales.*

## Carga del conjunto de datos en Pandas

Se importa la librería *pandas*, se carga el dataset y se muestran sus parámetros descriptivos incluyendo sus primeras filas, información general de tipos de datos, evaluación inicial de valores nulos, así como estadísticas descriptivas preliminares que otorga esta librería.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_morosidad = pd.read_csv("df_morosidad.csv")

In [3]:
df_temp = pd.read_csv("temp/CasoEspecialistaCampanas.csv")

## Limpieza y preprocesamiento de la información

En este análisis se va a revisar la calidad y consistencia de la información de entrada, incluyendo la posible existencia de valores nulos o en blanco, datos atípicos, entre otros.

In [ ]:
df_morosidad.head()

In [ ]:
df_morosidad.info()

In [ ]:
df_morosidad.describe()

### Verificación de posibles valores blancos o nulos

In [ ]:
df_morosidad.isna().sum()

Se puede ver en la celda anterior que de manera preliminar no se cuenta con valores nulos en el presente conjunto de datos, por lo cual se procede a verificar si hay valores nulos según el contexto del negocio así como valores inconsistentes.

### Verificación de posibles valores inconsistentes

Se realiza un análisis de los valores únicos por cada columna para determinar si podría resultar conveniente convertirlos al tipo categórico de pandas.

In [ ]:
df_morosidad.dtypes

In [ ]:
posibles_columnas_categoricas = [
    "rango_morosidad",
    "producto_host",
    "gestor",
    "recibe_sueldo_fijo",
    "region",
    "sector_general"
]

for col in posibles_columnas_categoricas:
    print(f'Valores únicos en "{col}":')
    print(df_morosidad[col].unique())
    print("-" * 50)

Se puede observar que no existen valores inconsistentes al haber analizado los valores únicos en cada columna las columnas o variables ***rango_morosidad***, ***producto_host***, ***gestor***, ***recibe_sueldo_fijo***, ***region*** y ***sector_general*** pueden convertirse a un tipo de variable categórico. Además la variable ***recibe_sueldo_fijo*** puede convertirse a binaria para un procesamiento más eficiente.

In [ ]:
df_morosidad[posibles_columnas_categoricas] = df_morosidad[posibles_columnas_categoricas].astype("category")
df_morosidad["recibe_sueldo_fijo"] = df_morosidad["recibe_sueldo_fijo"]\
    .map({"SI": True, "NO": False}).astype("bool")

In [ ]:
df_morosidad.dtypes

## Exploración de datos

Con el fin de ayudar a recuperar la cartera vencida del Banco, se realizan 2 tipos de campañas para ayudar a aliviar ya sea los intereses o el plazo de pago de las cuotas restantes. Estas campañas son:

1. **Diferimiento**: Es una medida mediante la cual una entidad financiera pospone temporalmente el pago de una o más cuotas de un crédito, sin que eso implique que el cliente ha incumplido. Esta campaña se aplicará en clientes que tengan entre 21 y 60 días en mora.
2. **Normalización**: Es un proceso mediante el cual una entidad financiera busca reconducir un crédito moroso a un estado "al día", generalmente a través de reestructuración, refinanciamiento o acuerdos de pago según la capacidad real del cliente. Esta campaña se aplicará en clientes que tengan entre 10 y 20 días en mora.

El siguiente paso consiste en crear una columna condicional que refleje el tipo de campaña que se va a ofertar a cada cliente según la cantidad de días que el mismo se mantenga en mora.

In [ ]:
condiciones = [
    (df_morosidad["dias_mora"] >= 10) & (df_morosidad["dias_mora"] <= 20),
    (df_morosidad["dias_mora"] >= 21) & (df_morosidad["dias_mora"] <= 60)
]

resultados = ["DIFERIMIENTO", "NORMALIZACIÓN"]
df_morosidad["tipo_campania"] = np.select(condiciones, resultados, default="NO_APLICA")

In [ ]:
df_morosidad["tipo_campania"] = df_morosidad["tipo_campania"].astype("category")

In [ ]:
df_morosidad.sample(5)

In [ ]:
df_morosidad["tipo_campania"].value_counts()

### Cleintes por tipo de campaña

In [ ]:
plt.figure(figsize=(4, 3))
sns.countplot(
    data=df_morosidad,
    x="tipo_campania"
)

plt.title("Clientes por tipo de campaña")
plt.ylabel("Cantidad de clientes")
plt.xlabel("Campaña")
plt.show()

### Análisis por Canal de Gestión

In [ ]:
plt.figure(figsize=(5.5, 3.5))

sns.countplot(
    data=df_morosidad,
    x="tipo_campania",
    hue="gestor"
)

plt.title("Clientes por campaña y canal de atención")
plt.ylabel("Cantidad de clientes")
plt.xlabel("Campaña")
plt.legend(title="Canal")
plt.show()

In [ ]:
pivot = df_morosidad.groupby(["tipo_campania", "gestor"], observed=True) \
["monto_vencido"].sum().unstack()
pivot.plot(kind="bar", stacked=True)
plt.title("Monto vencido por campaña y canal")
plt.ylabel("Monto vencido ($)")
plt.xlabel("Campaña")
plt.show()

In [ ]:
sns.countplot(
    data=df_morosidad,
    x="tipo_campania",
    hue="recibe_sueldo_fijo"
)
plt.title("Clientes con/sin sueldo fijo por campaña")
plt.ylabel("Cantidad de clientes")
plt.xlabel("Campaña")
plt.legend(title="Recibe sueldo fijo")
plt.show()

In [ ]:
sns.scatterplot(data=df_morosidad, x="dias_mora", y="monto_vencido", hue="tipo_campania")
plt.title("Días de mora vs Monto vencido")
plt.xlabel("Días en mora")
plt.ylabel("Monto vencido ($)")
plt.legend(title="Campaña")
plt.show()

In [ ]:
provincia_counts = df_morosidad["region"].value_counts().sort_values()
provincia_counts.plot(kind="barh", figsize=(8,6))
plt.title("Clientes por región")
plt.xlabel("Cantidad de clientes")
plt.ylabel("Región")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(
    data=df_morosidad,
    y="sector_general",
    hue="tipo_campania",
    order=df_morosidad["sector_general"].value_counts().index
)
plt.title("Clientes por sector económico y campaña")
plt.xlabel("Cantidad de clientes")
plt.ylabel("Sector económico")
plt.legend(title="Campaña")
plt.show()

In [ ]:
print("Total de clientes:", len(df_morosidad))
print("Total monto vencido: $", df_morosidad["monto_vencido"].sum())
print("Total intereses acumulados: $", df_morosidad["intereses"].sum())
print("Clientes con sueldo fijo:", df_morosidad[df_morosidad["recibe_sueldo_fijo"] == True].shape[0])